In [ ]:
import torch
print(torch.version.cuda)

In [ ]:
!pip install -U datasets

In [ ]:
!pip install torch

In [ ]:
!pip install torch torchvision deepspeed transformers tokenizers sentencepiece shortuuid accelerate peft bitsandbytes pydantic markdown2 numpy scikit-learn gradio requests httpx uvicorn fastapi einops einops-exts timm flash-attn

# Drive Mount and Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
import json
import torch
import random
import warnings
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from PIL import Image

In [ ]:
# 환경 설정
warnings.filterwarnings("ignore")
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
print("✅ Using device:", device)

✅ Using device: cuda


# (만약 Data랑 Json파일을 만들었다면)Data 추가한 버전 불러오기

In [ ]:
!git clone {private_repo}

Cloning into 'MobileVLM-Finetuning'...
remote: Enumerating objects: 5854, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 5854 (delta 31), reused 35 (delta 30), pack-reused 5815 (from 2)
Receiving objects: 100% (5854/5854), 922.16 MiB | 16.85 MiB/s, done.
Resolving deltas: 100% (105/105), done.
Updating files: 100% (14273/14273), done.


In [ ]:
#already done
#os.mkdir('/path/to/lib/MobileVLM/mobilevlm-1.pretrain')
#os.mkdir('/path/to/lib/MobileVLM/mobilevlm-2.finetune')
#os.mkdir('/path/to/lib/MobileVLM/mobilevlm-2.finetune-lora')

In [ ]:
#already done
#!wget https://github.com/Meituan-AutoML/MobileVLM/releases/download/v0.2/mobilevlmv1_3b_stage1_mm_projector.bin -P /path/to/lib/MobileVLM/mobilevlm-1.pretrain/

In [ ]:
#!ls

drive  MobileVLM  sample_data


# MobileVLM-Finetuning -> MobileVLM으로 파일 이름 변경 후 아래 셀 진행

In [ ]:
import sys
from pathlib import Path
os.chdir('/content/MobileVLM')

In [ ]:
!ls

assets	LICENSE      mobilevlm	requirements.txt  run_v1.sh
data	mobilellama  README.md	run.sh		  scripts


# MobileVLM git 불러오기

In [ ]:
#!git clone https://github.com/Meituan-AutoML/MobileVLM.git

# Data Loading and Preprocessing
## And Make json file

In [ ]:
# dataset = load_dataset("lmms-lab/SEED-Bench", split="test")
# image_train_dataset=dataset.filter(lambda example: example['data_type']=='image')

In [ ]:
#len(image_train_dataset)

14233

In [ ]:
#image_train_dataset[0]['answer']

'A'

In [ ]:
#os.makedirs("/content/MobileVLM/data/finetune", exist_ok=True)

In [ ]:
# for idx, example in enumerate(tqdm(image_train_dataset)):
#     image=example["image"][0]
#     image_path = os.path.join("/content/MobileVLM/data/finetune", f"{idx:05d}.jpg")
#     image.save(image_path)

100%|██████████| 14233/14233 [07:28<00:00, 31.75it/s]


In [ ]:
# data=[]

# for idx, item in enumerate(tqdm(image_train_dataset)):
#     if idx == 0: # multi-modal dialog data 필요
#         example = {
#             "id": str(idx).zfill(5),
#             "image": f"{idx:05d}.jpg",
#             "conversations": [
#                 {
#                     "from": "human",
#                     "value": "<image>\n" + "Answer only with one of: A, B, C, or D\n" + f"Question : {item['question']}\n" + f"A: {item['choice_a']}\n B: {item['choice_b']}\n C: {item['choice_c']}\n D: {item['choice_d']}\n Answer:"
#                 },
#                 {
#                     "from": "gpt",
#                     "value": item['answer']
#                 },
#                 {
#                     "from": "human",
#                     "value": "Question : What is this place?\n A: living room\n B: kitchen\n C: bathroom \n D: bedroom\n Answer:"
#                 },
#                 {
#                     "from": "gpt",
#                     "value": "C"
#                 }
#             ]
#         }
#     else:
#         example = {
#             "id": str(idx).zfill(5),
#             "image": f"{idx:05d}.jpg",
#             "conversations": [
#                 {
#                     "from": "human",
#                     "value": "<image>\n" + "Answer only with one of: A, B, C, or D\n" + f"Question : {item['question']}\n" + f"A: {item['choice_a']}\n B: {item['choice_b']}\n C: {item['choice_c']}\n D: {item['choice_d']}\n Answer:"
#                 },
#                 {
#                     "from": "gpt",
#                     "value": item['answer']
#                 }
#             ]
#         }
#     data.append(example)

100%|██████████| 14233/14233 [06:00<00:00, 39.45it/s]


In [ ]:
# len(data)

14233

In [ ]:
# data.append({
#     "id":str(len(image_train_dataset)).zfill(5),
#     "conversations":[ # text-only data 필요
#         {
#             "from": "human",
#             "value": "What is 1 + 1?\n A: 0\n B: 1\n C: 2\n D: 3\n"
#         },
#         {
#             "from": "gpt",
#             "value": "C"
#         }
#     ]
# })
# with open("data/finetune/seed_bench.json", "w", encoding="utf-8") as f:
#     json.dump(data, f)

# Json 파일과 Data Git에 업로드

# Memory

In [ ]:
#del dataset
#del image_train_dataset
import gc
gc.collect()

100

# Dependency Debugging

In [ ]:
!pip install --upgrade transformers==4.33.1

In [ ]:
!pip install --upgrade peft==0.10.0

In [ ]:
!pip install --upgrade accelerate==0.28.0

In [ ]:
!pip install --upgrade flash-attn==2.7.4.post1

# Finetuning With LoRA

In [ ]:
%%shell
export PYTHONPATH=/content/MobileVLM:$PYTHONPATH

echo ">>> Start Visual-Instruction Tuning..."
LANGUAGE_MODEL=mtgv/MobileLLaMA-2.7B-Chat
VISION_MODEL=openai/clip-vit-large-patch14-336
OUTPUT_DIR=/path/to/lib/MobileVLM
OUTPUT_DIR_PT=${OUTPUT_DIR}/mobilevlm-1.pretrain
OUTPUT_DIR_FT=${OUTPUT_DIR}/mobilevlm-2.finetune-lora
mkdir -p ${OUTPUT_DIR_FT}
deepspeed mobilevlm/train/train_mem.py \
    --deepspeed scripts/deepspeed/zero3.json \
    --lora_enable True --lora_r 8 --lora_alpha 16 \
    --learning_rate 2e-5 \
    --model_name_or_path ${LANGUAGE_MODEL} \
    --version v1 \
    --data_path data/finetune/seed_bench.json \
    --image_folder data/finetune \
    --vision_tower ${VISION_MODEL} \
    --vision_tower_type clip \
    --pretrain_mm_mlp_adapter ${OUTPUT_DIR_PT}/mobilevlmv1_3b_stage1_mm_projector.bin \
    --mm_projector_type ldpnet \
    --mm_vision_select_layer -2 \
    --mm_use_im_start_end False \
    --mm_use_im_patch_token False \
    --image_aspect_ratio pad \
    --group_by_modality_length True \
    --bf16 True \
    --output_dir ${OUTPUT_DIR_FT} \
    --num_train_epochs 3 \
    --per_device_train_batch_size 128 \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps 1 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 1000 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --tf32 True \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --dataloader_num_workers 4 \
    --lazy_preprocess True \
    --report_to none \
    2>&1 | tee -a ${OUTPUT_DIR_FT}/log.txt &&
python3 scripts/mergelora.py ${LANGUAGE_MODEL} ${OUTPUT_DIR}/mobilevlm-2.finetune-lora ${OUTPUT_DIR}/mobilevlm-2.finetune
echo "Done."

>>> Start Visual-Instruction Tuning...
[2025-07-20 15:13:54,208] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2025-07-20 15:13:58,176] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
2025-07-20 15:13:58.824528: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753024438.846181    8820 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753024438.852888    8820 cuda_blas.cc:1418] Unable to 